# Team Rush - Hand Gesture Recognition

In [ ]:
import cv2
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import math
import random

In [ ]:
folder_path = "E:/UiS/DAT540/Group Project/images/"
# The folders Need to be such that -
    # 1 finger = one
    # 2 fingers = two
    # And so on
    # A zip file of the images folders is attached with the project submission.

In [ ]:
# Loading files and putting them in lists
images = [cv2.imread(file) for file in glob.glob(folder_path + 'zero/*.jpg')]
images_real_results = [0 for file in glob.glob(folder_path + 'zero/*.jpg')]

images.extend([cv2.imread(file) for file in glob.glob(folder_path + 'one/*.jpg')])
images_real_results.extend([1 for file in glob.glob(folder_path + 'one/*.jpg')])

images.extend([cv2.imread(file) for file in glob.glob(folder_path + 'two/*.jpg')])
images_real_results.extend([2 for file in glob.glob(folder_path + 'two/*.jpg')])

images.extend([cv2.imread(file) for file in glob.glob(folder_path + 'three/*.jpg')])
images_real_results.extend([3 for file in glob.glob(folder_path + 'three/*.jpg')])

images.extend([cv2.imread(file) for file in glob.glob(folder_path + 'four/*.jpg')])
images_real_results.extend([4 for file in glob.glob(folder_path + 'four/*.jpg')])

images.extend([cv2.imread(file) for file in glob.glob(folder_path + 'five/*.jpg')])
images_real_results.extend([5 for file in glob.glob(folder_path + 'five/*.jpg')])

In [ ]:
# First shuffle the data
# Since we have 2 lists and they reflect each others values,
# we had to combine them so that we can shuffle the values together,
# and finally separate them put them in their respective lists.
combined = list(zip(images, images_real_results))
random.shuffle(combined)
images[:], images_real_results[:] = zip(*combined)

# Separating the data into Training and Test data
# For this project we have separated them in a 66:34 ratio
split_index = int((66/100)*len(images))

training_images = images[:split_index]
training_image_results = images_real_results[:split_index]

test_images = images[split_index:]
test_image_results = images_real_results[split_index:]

print("No. of Training Images:",len(training_images))
print("No. of Test Images:", len(test_images))

In [ ]:
# Function to find the number of defects in an image
def find_number_of_defects(img):
    blur = cv2.GaussianBlur(img, (3,3), 0)
    
    # Change color-space from BGR -> HSV
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)

    # Create a binary image with where white will be skin colors and rest is black
    mask2 = cv2.inRange(hsv, np.array([2,0,0]), np.array([20,255,255]))
    # Kernel for morphological transformation    
    kernel = np.ones((5,5))

    # Apply morphological transformations to filter out the background noise
    dilation = cv2.dilate(mask2, kernel, iterations = 1)
    erosion = cv2.erode(dilation, kernel, iterations = 1)    

    # Apply Gaussian Blur and Threshold
    filtered = cv2.GaussianBlur(erosion, (3,3), 0)
    ret,thresh = cv2.threshold(filtered, 127, 255, 0)

    #Find Contours
    _,contours,hierarchy= cv2.findContours(filtered,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    #Find contour of max area(hand)
    cnt = max(contours, key = lambda x: cv2.contourArea(x))
    
    #Approx the contour a little
    epsilon = 0.0005*cv2.arcLength(cnt,True)
    approx= cv2.approxPolyDP(cnt,epsilon,True)

    #Make convex hull around hand
    hull = cv2.convexHull(cnt)

    #define area of hull and area of hand
    areahull = cv2.contourArea(hull)
    areacnt = cv2.contourArea(cnt)

    #find the percentage of area not covered by hand in convex hull
    arearatio=((areahull-areacnt)/areacnt)*100

    #find the defects in convex hull with respect to hand
    hull = cv2.convexHull(approx, returnPoints=False)
    defects = cv2.convexityDefects(approx, hull)
    
     #code for finding no. of defects due to fingers
    l=0
    for i in range(defects.shape[0]):
        s,e,f,d = defects[i,0]
        start = tuple(approx[s][0])
        end = tuple(approx[e][0])
        far = tuple(approx[f][0])
        pt= (100,180)

       # find length of all sides of triangle
        a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
        b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
        c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
        s = (a+b+c)/2
        ar = math.sqrt(s*(s-a)*(s-b)*(s-c))

            #distance between point and convex hull
        d=(2*ar)/a

            # apply cosine rule here
        angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57


            # ignore angles > 90 and ignore points very close to convex hull(they generally come due to noise)
        if angle <= 90 and d>30:
                l += 1
        cv2.circle(img, far, 3, [255,0,0], -1)

            #draw lines around hand
        cv2.line(img,start, end, [0,255,0], 2)
    
    #l+=1

    if l == 0:
        if areacnt < 2000:
            l = -1
            #print('No fingers detected')
        else:
            if arearatio < 12:
                l = -1
                #print('0')
            elif arearatio < 17.5:
                l = 0
                #print('Best of luck')
            else:
                l = 0
                #print('1')
    #elif l==2:
        #print('2')
    #elif l==3:
        #print(3)
    #elif l==4:
        #print('4')
    #elif l ==5:
        #print('5')
        
    return l

In [ ]:
#Collecting images from the directory

n_defects = []
#cv_img = []

for img in training_images:
    n_defects.append(find_number_of_defects(img))

# Output
#fmt = '{:<8}{:<20}{}'
#print(fmt.format('', 'Defects', 'Number of Fingers'))
#for i, (defect, no_of_fingers) in enumerate(zip(n_defects, training_image_results)):
#    print(fmt.format(i+1, defect, no_of_fingers))

In [ ]:
# Importing KNN from Scikit-Learn
from sklearn.neighbors import KNeighborsClassifier

features = list(zip(n_defects))
    
model = KNeighborsClassifier(n_neighbors = 3)
model.fit(features, training_image_results)

In [ ]:
# Function for predicting with KNN
def knn_prediction(predict_for):
    prediction = model.predict([[predict_for]]) # Prediction with 'n' defects
    return prediction[0]

In [ ]:
print("Number of fingers:", knn_prediction(predict_for = 1) ) # KNN Prediction for 1 defect

In [ ]:
print("Number of fingers:", knn_prediction(predict_for = 3) ) # KNN Prediction for 3 defects

In [ ]:
# The Accuracy of the model

#fmt = '{:<8}{:<20}{:<20}{}'
#print(fmt.format('', 'Defects', 'Prediction', 'Real Number of Fingers'))
correct_count = 0

for i in range(len(test_images)):
    defect = find_number_of_defects(test_images[i])

    prediction = knn_prediction(predict_for = defect)

    #print(fmt.format(i+1, defect, prediction, test_image_results[i]))

    if(prediction == test_image_results[i]):
        correct_count = correct_count + 1
        
print("Total test images:",len(test_images))
print("Number of correct predictions:",correct_count)
print("\nAccuracy:", 100*(correct_count/len(test_images)))


In [ ]:
captured_image = None

cam = cv2.VideoCapture(0)

cv2.namedWindow("hand gesture")

img_counter = 0

while cam.isOpened():
    ret, frame = cam.read()
    
    cv2.rectangle(frame,(70,70),(300,300),(100,200,50),0)
    crop_image = frame[90:300, 90:300]
    
    cv2.imshow("hand gesture", frame)
    
    if not ret:
        break
    k = cv2.waitKey(1)

    if k%256 == 27:
        # ESC pressed
        break
    elif k%256 == 32: # SPACE pressed
        captured_image = crop_image
        break
        
cam.release()

cv2.destroyAllWindows()

# Predict the Number of Fingers in the Captured Image.
defect = find_number_of_defects(captured_image)

prediction = knn_prediction(predict_for = defect)

fmt = '{:<20}{}'
print(fmt.format('Defects Found', 'Prediction'))
print(fmt.format(defect, prediction))
#plt.imshow(captured_image)